<a href="https://colab.research.google.com/github/VitorNunesCPT/LinguagemFormaisEAutomatos/blob/main/p3LFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINGUAGENS FORMAIS E AUTÔMATOS (CP)

Docente: IVO JOSE DA CUNHA SERRA

Discentes:  
ANTONIO FIALHO DA SILVA NETO

EMANOEL SILVA LIMA

VITOR FERREIRA NUNES

Trabalho para a 3a avaliação


1) Implementação do algoritmo de transformação AFN -> AFD.

Restrições:
O AFN de entrada possuirá no máximo: 4 estados, 8 transições e 3 símbolos do
alfabeto.


In [ ]:
from collections import defaultdict, deque

def afn_para_afd(estados, estado_inicial, funcao_programa, estados_finais):
    # Convertendo entrada para formatos adequados
    estados = estados.split(', ')
    estado_inicial = int(estado_inicial)
    estados_finais = set(map(int, estados_finais.split(', ')))

    # Inicializando estrutura de dados para transições do AFN
    transicoes = defaultdict(lambda: defaultdict(set))
    for transicao in funcao_programa:
        origem, simbolo, destino = int(transicao[0]), transicao[1], int(transicao[2])
        transicoes[origem][simbolo].add(destino)

    # Função para calcular o fechamento epsilon de um estado
    def fecho_epsilon(estado):
        stack = [estado]
        fecho = {estado}
        while stack:
            atual = stack.pop()
            # Adiciona todos os estados alcançáveis por transições epsilon
            for prox in transicoes[atual].get('', []):
                if prox not in fecho:
                    fecho.add(prox)
                    stack.append(prox)
        return fecho

    # Função para mover-se através de transições com um símbolo específico
    def mover(estados, simbolo):
        proximos_estados = set()
        for estado in estados:
            proximos_estados.update(transicoes[estado].get(simbolo, []))
        return proximos_estados

    # Inicialização do estado inicial do AFD como o fecho epsilon do estado inicial do AFN
    novo_estado_inicial = frozenset(fecho_epsilon(estado_inicial))
    estados_afd = {novo_estado_inicial}  # Conjunto de estados do AFD
    fila = deque([novo_estado_inicial])  # Fila para processamento dos novos estados do AFD
    transicoes_afd = {}  # Estrutura para armazenar as transições do AFD
    novos_estados_finais = set()  # Conjunto de estados finais do AFD

    # Verifica se o estado inicial do AFN é também um estado final do AFD
    if novo_estado_inicial & estados_finais:
        novos_estados_finais.add(novo_estado_inicial)

    # Construção do AFD
    while fila:
        estado_atual = fila.popleft()
        transicoes_afd[estado_atual] = {}
        simbolos = {transicao[1] for transicao in funcao_programa if transicao[1]}

        for simbolo in simbolos:
            # Calcula o próximo estado do AFD
            prox_estado = frozenset().union(*[frozenset(fecho_epsilon(est)) for est in mover(estado_atual, simbolo)])

            if prox_estado:
                transicoes_afd[estado_atual][simbolo] = prox_estado  # Define a transição no AFD

                if prox_estado not in estados_afd:
                    estados_afd.add(prox_estado)  # Adiciona novo estado ao conjunto de estados do AFD
                    fila.append(prox_estado)  # Adiciona novo estado à fila para processamento

                if prox_estado & estados_finais:
                    novos_estados_finais.add(prox_estado)  # Marca o estado como final se contém estado final do AFN

    # Função para formatar estado como um conjunto entre chaves
    def formatar_estado(estado):
        return "{" + ", ".join(sorted(map(str, estado))) + "}"

    # NOVA FUNÇÃO: Criar e exibir tabela de transições
    def exibir_tabela_transicoes(estados_afd, transicoes_afd, simbolos):
        print("\n=== TABELA DE TRANSIÇÕES DO AFD ===")

        # Obter lista ordenada de estados e símbolos
        estados_ordenados = sorted(estados_afd, key=lambda x: sorted(x))
        simbolos_ordenados = sorted(simbolos)

        # Cabeçalho da tabela
        cabecalho = "Estado".ljust(15)
        for simbolo in simbolos_ordenados:
            cabecalho += f"| {simbolo}".ljust(15)
        print(cabecalho)
        print("-" * len(cabecalho))

        # Linhas da tabela
        for estado in estados_ordenados:
            estado_str = formatar_estado(estado).ljust(15)
            linha = estado_str

            for simbolo in simbolos_ordenados:
                if simbolo in transicoes_afd.get(estado, {}):
                    destino = formatar_estado(transicoes_afd[estado][simbolo])
                    linha += f"| {destino}".ljust(15)
                else:
                    linha += "| ∅".ljust(15)  # Estado vazio para transições não definidas

            print(linha)
        print("=" * len(cabecalho))

    # Formatação dos resultados para impressão
    estados_afd_str = {formatar_estado(s) for s in estados_afd}
    transicoes_afd_str = []

    # Formatando transições do AFD para impressão
    for origem, transicoes in transicoes_afd.items():
        origem_str = formatar_estado(origem)
        for simbolo, destino in transicoes.items():
            destino_str = formatar_estado(destino)
            transicoes_afd_str.append(f"{origem_str} --{simbolo}--> {destino_str}")

    # Formatando estados finais do AFD para impressão
    estados_finais_str = {formatar_estado(s) for s in novos_estados_finais}

    # Obter símbolos para a tabela
    simbolos = {transicao[1] for transicao in funcao_programa if transicao[1]}

    # Imprimindo os resultados formatados na tela
    print("....... AFD Convertido ............")
    print("Estados do AFD:", ", ".join(estados_afd_str))
    print("Estado inicial do AFD:", formatar_estado(novo_estado_inicial))
    print("Função programa do AFD:")
    for transicao in transicoes_afd_str:
        print(transicao)
    print("Estados finais do AFD:", ", ".join(estados_finais_str))

    # NOVA CHAMADA: Exibir tabela de transições
    exibir_tabela_transicoes(estados_afd, transicoes_afd, simbolos)

    print("..................................................")

# Código principal para entrada de dados e chamada da função de conversão
print("Informe os estados do autômato (máximo 4 estados):")
estados = input()
if len(estados.split(', ')) > 4:
    print("Erro: número máximo de estados excedido.")
    exit()

print("Informe o estado inicial:")
estado_inicial = input()

print("Informe a função programa (máximo 8 transições, pressione Enter duas vezes para finalizar):")
funcao_programa = []
simbolos_alfabeto = set()
while True:
    entrada = input()
    if entrada == "":
        break
    entrada_split = entrada.split()
    if len(entrada_split) != 3:
        print("Formato inválido. Por favor, use o formato 'estado simbolo estado'. Exemplo: 0 a 2")
        continue
    try:
        entrada_convertida = [int(entrada_split[0]), entrada_split[1], int(entrada_split[2])]
        simbolos_alfabeto.add(entrada_split[1])
        if len(simbolos_alfabeto) > 3:
            print("Erro: número máximo de símbolos do alfabeto excedido.")
            exit()
        funcao_programa.append(entrada_convertida)
        if len(funcao_programa) > 8:
            print("Erro: número máximo de transições excedido.")
            exit()
    except ValueError:
        print("Erro: certifique-se de que os estados sejam números inteiros e o símbolo seja um caractere. Exemplo: 0 a 2")

print("Informe os estados finais:")
estados_finais = input()
print("\n")

# Chamada da função principal para converter AFN para AFD e imprimir o resultado
afn_para_afd(estados, estado_inicial, funcao_programa, estados_finais)

Informe os estados do autômato (máximo 4 estados):
0, 1, 2, 3
Informe o estado inicial:
0
Informe a função programa (máximo 8 transições, pressione Enter duas vezes para finalizar):
0 a 2
0 a 3
2 b 1
1 b 0

Informe os estados finais:
0, 3


....... AFD Convertido ............
Estados do AFD: {0}, {1}, {2, 3}
Estado inicial do AFD: {0}
Função programa do AFD:
{0} --a--> {2, 3}
{2, 3} --b--> {1}
{1} --b--> {0}
Estados finais do AFD: {0}, {2, 3}

=== TABELA DE TRANSIÇÕES DO AFD ===
Estado         | a            | b            
---------------------------------------------
{0}            | {2, 3}       | ∅            
{1}            | ∅            | {0}          
{2, 3}         | ∅            | {1}          
..................................................
